In [5]:
#libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)


In [6]:
# !pip install pyreadstat   # Levanta archivos .sav

### Levanto ENMODO 2009
https://www.argentina.gob.ar/transporte/dgppse/publicaciones/encuestas

In [7]:
import pyreadstat

Hogares_data, Hogares_meta = pyreadstat.read_sav("ENMODO_HOGARES_pub_20121115.sav", encoding = 'latin1')
Personas_data, Personas_meta = pyreadstat.read_sav("ENMODO_PERSONAS_pub_20121115.sav", encoding = 'latin1')
Viajes_data, Viajes_meta = pyreadstat.read_sav("ENMODO_VIAJES_pub_20121115.sav", encoding = 'latin1')
Etapas_data, Etapas_meta = pyreadstat.read_sav("ENMODO_ETAPAS_pub_20121115.sav", encoding = 'latin1')

Hogares = pyreadstat.set_value_labels(Hogares_data, Hogares_meta, formats_as_category=True)
Personas = pyreadstat.set_value_labels(Personas_data, Personas_meta, formats_as_category=True)
Viajes = pyreadstat.set_value_labels(Viajes_data, Viajes_meta, formats_as_category=True)
Etapas = pyreadstat.set_value_labels(Etapas_data, Etapas_meta, formats_as_category=True)


Etapas['MODOTRAN'] = Etapas.MODOTRAN.astype(str)  
Etapas = Etapas.rename(columns={'MODOTRAN': 'Modo'})

### Creo zonas CABA, Conurbano

In [8]:
Hogares['Zona'] = 'Conurbano'
Hogares.loc[Hogares.PARTIDO.str[:9] == 'Cap. Fed.', 'Zona'] = 'CABA'

In [9]:
Personas['Zona'] = 'Conurbano'
Personas.loc[Personas.PARTIDO.str[:9] == 'Cap. Fed.', 'Zona'] = 'CABA'

PoblTotal= Personas.wt1.sum().round().astype(int)
PoblConurbano = Personas[Personas.Zona!='CABA'].wt1.sum().round().astype(int)
PoblCABA      = Personas[Personas.Zona=='CABA'].wt1.sum().round().astype(int)

In [10]:
# Viajes total, CABA, Conurbano
Viajes['Zona'] = 'Conurbano'
Viajes.loc[Viajes.partido.str[:9] == 'Cap. Fed.', 'Zona'] = 'CABA'

Viajes['Zona_destino'] = 'Destino Conurbano'
Viajes.loc[Viajes.TT_PARTIDO_DESTINO.str[:9] == 'Cap. Fed.', 'Zona_destino'] = 'Destino CABA'

Viajes['Zona_origen'] = 'Origen Conurbano'
Viajes.loc[Viajes.TT_PARTIDO_ORIGEN.str[:9] == 'Cap. Fed.', 'Zona_origen'] = 'Origen CABA'

In [11]:
# Etapas total, CABA, Conurbano
Etapas['Zona'] = 'Conurbano'
Etapas.loc[Etapas.partido.str[:9] == 'Cap. Fed.', 'Zona'] = 'CABA'




### Clasifico modos

In [12]:
Etapas.Modo.unique()

array(['Ferrocarril', 'Subte / Premetro', 'A pie',
       'Automovil / conductor', 'Taxi', 'Automovil / acompañante',
       'Colectivo', 'Bicicleta', 'Remis', 'Charter / combi',
       'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar',
       'Lancha Colectiva'], dtype=object)

In [13]:
Etapas['Modo'] = Etapas['Modo'].replace(['Ferrocarril', 'Subte / Premetro', 'A pie', 'Automovil / conductor', 'Taxi', 'Automovil / acompañante',
       'Colectivo', 'Bicicleta', 'Remis', 'Charter / combi', 'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva'],
        ['Ferrocarril', 'Subte/Premetro', 'A pie', 'Automovil/conductor', 'Taxi', 'Automovil/acompañante', 'Colectivo', 'Bicicleta', 'Remis', 'Charter/combi',
       'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva'])

In [14]:
# Creo sub-categorías de agrupación en Etapas

etapas_cats_old = ['Ferrocarril', 'Subte/Premetro', 'A pie', 'Automovil/conductor', 'Taxi', 'Automovil/acompañante', 'Colectivo', 'Bicicleta',
               'Remis', 'Charter/combi', 'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva']
etapas_cats_new = ['Público', 'Público', 'No Motorizado', 'Auto/Moto', 'A demanda', 'Auto/Moto', 'Público', 'No Motorizado',
               'A demanda', 'A demanda', 'A demanda', 'Auto/Moto', 'A demanda', 'A demanda', 'A demanda']


Etapas['Modo_agg'] = Etapas['Modo'].replace(etapas_cats_old, etapas_cats_new)

# Creo variables dummies para agregación de etapas a nivel de viaje
Etapas = pd.concat([Etapas, pd.get_dummies(Etapas.Modo)], axis=1)
Etapas = pd.concat([Etapas, pd.get_dummies(Etapas.Modo_agg)], axis=1)

### Análisis de Etapas de viajes
La tabla de Etapas de viajes incluye todos los modos de transporte utilizados para un viaje

In [15]:
particion_etapas = Etapas.groupby(['Modo', 'Modo_agg']).wt1.sum().round().astype(int).reset_index()
particion_etapas.columns = ['Modo', 'Modo_agg', 'Etapas']
particion_etapas['Etapas_porc'] = (particion_etapas.Etapas / particion_etapas.Etapas.sum() * 100).round(1)

particion_etapas_caba = Etapas[Etapas.Zona=='CABA'].groupby(['Modo', 'Modo_agg']).wt1.sum().round().astype(int).reset_index()
particion_etapas_caba.columns = ['Modo', 'Modo_agg', 'CABA']
particion_etapas_caba['CABA_porc'] = (particion_etapas_caba.CABA / particion_etapas_caba.CABA.sum() * 100).round(1)

particion_etapas_conurbano = Etapas[Etapas.Zona!='CABA'].groupby(['Modo', 'Modo_agg']).wt1.sum().round().astype(int).reset_index()
particion_etapas_conurbano.columns = ['Modo', 'Modo_agg', 'Conurbano']
particion_etapas_conurbano['Conurbano_porc'] = (particion_etapas_conurbano.Conurbano / particion_etapas_conurbano.Conurbano.sum() * 100).round(1)

particion_etapas = particion_etapas.set_index(['Modo', 'Modo_agg']).join(particion_etapas_caba.set_index(['Modo', 'Modo_agg'])).reset_index()
particion_etapas = particion_etapas.set_index(['Modo', 'Modo_agg']).join(particion_etapas_conurbano.set_index(['Modo', 'Modo_agg'])).reset_index()

particion_etapas = particion_etapas.fillna(0)

display(particion_etapas.sort_values('Etapas', ascending=False))

particion_etapas.groupby('Modo_agg').sum().reset_index().sort_values('Etapas', ascending=False)

,Modo,Modo_agg,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
6,Colectivo,Público,8758214,39.1,2020916.0,37.1,6737299,39.7
0,A pie,No Motorizado,5420342,24.2,1355707.0,24.9,4064635,24.0
2,Automovil/conductor,Auto/Moto,2663308,11.9,703343.0,12.9,1959965,11.6
1,Automovil/acompañante,Auto/Moto,1429379,6.4,357179.0,6.6,1072200,6.3
7,Ferrocarril,Público,1404053,6.3,122733.0,2.3,1281320,7.6
12,Subte/Premetro,Público,818687,3.7,484097.0,8.9,334590,2.0
3,Bicicleta,No Motorizado,672156,3.0,66664.0,1.2,605492,3.6
11,Remis,A demanda,366623,1.6,40184.0,0.7,326439,1.9
9,Motocicleta,Auto/Moto,257424,1.1,34974.0,0.6,222450,1.3
13,Taxi,A demanda,216591,1.0,197054.0,3.6,19537,0.1


,Modo_agg,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
3,Público,10980954,49.1,2627746.0,48.3,8353209,49.3
2,No Motorizado,6092498,27.2,1422371.0,26.1,4670127,27.6
1,Auto/Moto,4350111,19.4,1095496.0,20.1,3254615,19.2
0,A demanda,991203,4.5,301810.0,5.4,689393,4.1


### Etapas en modos no motorizados

In [16]:
particion_etapas1 = particion_etapas[particion_etapas.Modo_agg != 'No Motorizado'].reset_index(drop=True)
particion_etapas1['Etapas_porc'] = (particion_etapas1['Etapas'] / particion_etapas1['Etapas'].sum() * 100).round(1)
particion_etapas1['CABA_porc'] = (particion_etapas1['CABA'] / particion_etapas1['CABA'].sum() * 100).round(1)
particion_etapas1['Conurbano_porc'] = (particion_etapas1['Conurbano'] / particion_etapas1['Conurbano'].sum() * 100).round(1)

display(particion_etapas1.sort_values('Etapas', ascending=False))

particion_etapas1.groupby('Modo_agg').sum().sort_values('Etapas', ascending=False)

,Modo,Modo_agg,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
4,Colectivo,Público,8758214,53.7,2020916.0,50.2,6737299,54.8
1,Automovil/conductor,Auto/Moto,2663308,16.3,703343.0,17.5,1959965,15.9
0,Automovil/acompañante,Auto/Moto,1429379,8.8,357179.0,8.9,1072200,8.7
5,Ferrocarril,Público,1404053,8.6,122733.0,3.0,1281320,10.4
10,Subte/Premetro,Público,818687,5.0,484097.0,12.0,334590,2.7
9,Remis,A demanda,366623,2.2,40184.0,1.0,326439,2.7
7,Motocicleta,Auto/Moto,257424,1.6,34974.0,0.9,222450,1.8
11,Taxi,A demanda,216591,1.3,197054.0,4.9,19537,0.2
12,Transporte escolar,A demanda,208318,1.3,39340.0,1.0,168978,1.4
3,Charter/combi,A demanda,106165,0.7,10690.0,0.3,95475,0.8


,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
Modo_agg,,,,,,
Público,10980954,67.3,2627746.0,65.2,8353209,67.9
Auto/Moto,4350111,26.7,1095496.0,27.3,3254615,26.4
A demanda,991203,6.1,301810.0,7.6,689393,5.7


### Etapas en modos de transporte público

In [17]:
particion_etapas2 = particion_etapas[particion_etapas.Modo_agg == 'Público'].reset_index(drop=True)
particion_etapas2['Etapas_porc'] = (particion_etapas2['Etapas'] / particion_etapas2['Etapas'].sum() * 100).round(1)
particion_etapas2['CABA_porc'] = (particion_etapas2['CABA'] / particion_etapas2['CABA'].sum() * 100).round(1)
particion_etapas2['Conurbano_porc'] = (particion_etapas2['Conurbano'] / particion_etapas2['Conurbano'].sum() * 100).round(1)

particion_etapas2.to_csv('Etapas_tpubl2009.csv', sep=';', encoding='latin1')
particion_etapas2

,Modo,Modo_agg,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
0,Colectivo,Público,8758214,79.8,2020916.0,76.9,6737299,80.7
1,Ferrocarril,Público,1404053,12.8,122733.0,4.7,1281320,15.3
2,Subte/Premetro,Público,818687,7.5,484097.0,18.4,334590,4.0


# Análisis de viajes
La tabla de viajes incluye todos los viajes realizados por cada una de las pesonas que viaja. 
Cuando el viaje incluye varios modos de transporte se categoriza como Público Multietapa (si el viaje incluye una etapa de transporte público) o Otros Multietapa cuando no incluye etapa de transporte público

#### Incluyo modos de transporte de etapas en tabla de viajes

In [18]:
Viajes_agg = Etapas.groupby(['IDH', 'IDP', 'IDV']).agg({'Modo':'max', 'Modo_agg':'max', 'Ferrocarril':'sum', 'Subte/Premetro':'sum', 'A pie':'sum', 'Automovil/conductor':'sum', 'Taxi':'sum', 
                                                  'Automovil/acompañante':'sum', 'Colectivo':'sum', 'Bicicleta':'sum', 'Remis':'sum', 'Charter/combi':'sum', 
                                                  'Bus empresa':'sum', 'Motocicleta':'sum', 'Otros':'sum', 'Transporte escolar':'sum', 'Lancha Colectiva':'sum', 
                                                  'Público':'sum', 'No Motorizado':'sum', 'Auto/Moto':'sum', 'A demanda':'sum', 'wt1':'max'}).reset_index()

Viajes_agg['Cant_etapas'] = Viajes_agg[['Público', 'No Motorizado', 'Auto/Moto', 'A demanda']].sum(axis=1)
Viajes_agg.loc[(Viajes_agg.Cant_etapas>1)&(Viajes_agg.Público>0), 'Modo'] = 'Multietapa'
Viajes_agg.loc[(Viajes_agg.Cant_etapas>1)&((Viajes_agg.Modo!='Multietapa')), 'Modo'] = 'Otros Multietapa'
Viajes_agg.loc[Viajes_agg.Modo=='Otros Multietapa', 'Modo_agg'] = 'Otros Multietapa'
Viajes_agg.loc[Viajes_agg.Modo=='Multietapa', 'Modo_agg'] = 'Multietapa'

Viajes_agg['Modo_agg2'] = Viajes_agg.Modo_agg.replace(['No Motorizado', 'Auto/Moto', 'A demanda', 'Público',
       'Otros Multietapa', 'Multietapa'], ['No Motorizado', 'Auto/Moto', 'A demanda', 'Público',
       'A demanda', 'Público'])

##### Incluyo modo de transporte en tabla de viajes

In [19]:
Viajes = Viajes.set_index(['IDH', 'IDP', 'IDV']).join(Viajes_agg[['IDH', 'IDP', 'IDV', 'Modo', 'Modo_agg', 'Modo_agg2']].set_index(['IDH', 'IDP', 'IDV'])).reset_index()

### Viajes (Todos los modos)

In [20]:
particion_viajes = Viajes.groupby(['Modo', 'Modo_agg', 'Modo_agg2']).wt1.sum().round().astype(int).reset_index()
particion_viajes.columns = ['Modo', 'Modo_agg', 'Modo_agg2', 'Viajes']
particion_viajes['Viajes_porc'] = (particion_viajes.Viajes / particion_viajes.Viajes.sum() * 100).round(1)

particion_viajes_caba = Viajes[Viajes.Zona=='CABA'].groupby(['Modo', 'Modo_agg', 'Modo_agg2']).wt1.sum().round().astype(int).reset_index()
particion_viajes_caba.columns = ['Modo', 'Modo_agg', 'Modo_agg2', 'CABA']
particion_viajes_caba['CABA_porc'] = (particion_viajes_caba.CABA / particion_viajes_caba.CABA.sum() * 100).round(1)

particion_viajes_conurbano = Viajes[Viajes.Zona=='Conurbano'].groupby(['Modo', 'Modo_agg', 'Modo_agg2']).wt1.sum().round().astype(int).reset_index()
particion_viajes_conurbano.columns = ['Modo', 'Modo_agg', 'Modo_agg2', 'Conurbano']
particion_viajes_conurbano['Conurbano_porc'] = (particion_viajes_conurbano.Conurbano / particion_viajes_conurbano.Conurbano.sum() * 100).round(1)


particion_viajes = particion_viajes.set_index(['Modo', 'Modo_agg', 'Modo_agg2']).join(particion_viajes_caba.set_index(['Modo', 'Modo_agg', 'Modo_agg2'])).reset_index()
particion_viajes = particion_viajes.set_index(['Modo', 'Modo_agg', 'Modo_agg2']).join(particion_viajes_conurbano.set_index(['Modo', 'Modo_agg', 'Modo_agg2'])).reset_index()

print('Viajes según modos de transporte (Todos)')

display(particion_viajes.sort_values('Viajes', ascending=False))

display(particion_viajes.groupby('Modo_agg').sum().round(1).sort_values('Viajes' , ascending=False))

display(particion_viajes.groupby('Modo_agg2').sum().round(1).sort_values('Viajes' , ascending=False))

particion_viajes.to_csv('modos2009.csv', sep=';', encoding='latin1')

Viajes según modos de transporte (Todos)


,Modo,Modo_agg,Modo_agg2,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
6,Colectivo,Público,Público,5670340,28.7,1655998,32.2,4014342,27.4
0,A pie,No Motorizado,No Motorizado,5419601,27.4,1355707,26.4,4063894,27.8
2,Automovil/conductor,Auto/Moto,Auto/Moto,2650057,13.4,699240,13.6,1950818,13.3
9,Multietapa,Multietapa,Público,2178389,11.0,296453,5.8,1881937,12.9
1,Automovil/acompañante,Auto/Moto,Auto/Moto,1366344,6.9,343476,6.7,1022868,7.0
3,Bicicleta,No Motorizado,No Motorizado,629467,3.2,64297,1.3,565170,3.9
13,Subte/Premetro,Público,Público,372489,1.9,359696,7.0,12793,0.1
7,Ferrocarril,Público,Público,349324,1.8,45958,0.9,303366,2.1
12,Remis,A demanda,A demanda,331026,1.7,38866,0.8,292160,2.0
8,Motocicleta,Auto/Moto,Auto/Moto,255203,1.3,34974,0.7,220229,1.5


,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
Modo_agg,,,,,,
Público,6392153,32.4,2061652,40.1,4330501,29.6
No Motorizado,6049068,30.6,1420004,27.7,4629064,31.7
Auto/Moto,4271604,21.6,1077690,21.0,3193915,21.8
Multietapa,2178389,11.0,296453,5.8,1881937,12.9
A demanda,862182,4.4,279859,5.5,582323,4.1
Otros Multietapa,14586,0.1,1876,0.0,12711,0.1


,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
Modo_agg2,,,,,,
Público,8570542,43.4,2358105,45.9,6212438,42.5
No Motorizado,6049068,30.6,1420004,27.7,4629064,31.7
Auto/Moto,4271604,21.6,1077690,21.0,3193915,21.8
A demanda,876768,4.5,281735,5.5,595034,4.2


### Viajes no motorizados

In [21]:
print('Viajes (modos motorizados)')
particion_viajes2 = particion_viajes[particion_viajes.Modo_agg2!='No Motorizado'].copy().reset_index(drop=True)
particion_viajes2['Viajes_porc'] = (particion_viajes2['Viajes'] / particion_viajes2['Viajes'].sum() * 100).round(1)
particion_viajes2['CABA_porc'] = (particion_viajes2['CABA'] / particion_viajes2['CABA'].sum() * 100).round(1)
particion_viajes2['Conurbano_porc'] = (particion_viajes2['Conurbano'] / particion_viajes2['Conurbano'].sum() * 100).round(1)
particion_viajes2 = particion_viajes2.sort_values('Viajes', ascending=False).reset_index(drop=True)
particion_viajes2.to_csv('modos_publico2009.csv', sep=';', encoding='latin1')
display(particion_viajes)
particion_viajes.groupby('Modo_agg').sum().sort_values('Viajes', ascending=False)


Viajes (modos motorizados)


,Modo,Modo_agg,Modo_agg2,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
0,A pie,No Motorizado,No Motorizado,5419601,27.4,1355707,26.4,4063894,27.8
1,Automovil/acompañante,Auto/Moto,Auto/Moto,1366344,6.9,343476,6.7,1022868,7.0
2,Automovil/conductor,Auto/Moto,Auto/Moto,2650057,13.4,699240,13.6,1950818,13.3
3,Bicicleta,No Motorizado,No Motorizado,629467,3.2,64297,1.3,565170,3.9
4,Bus empresa,A demanda,A demanda,18318,0.1,2731,0.1,15587,0.1
5,Charter/combi,A demanda,A demanda,59941,0.3,7196,0.1,52745,0.4
6,Colectivo,Público,Público,5670340,28.7,1655998,32.2,4014342,27.4
7,Ferrocarril,Público,Público,349324,1.8,45958,0.9,303366,2.1
8,Motocicleta,Auto/Moto,Auto/Moto,255203,1.3,34974,0.7,220229,1.5
9,Multietapa,Multietapa,Público,2178389,11.0,296453,5.8,1881937,12.9


,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
Modo_agg,,,,,,
Público,6392153,32.4,2061652,40.1,4330501,29.6
No Motorizado,6049068,30.6,1420004,27.7,4629064,31.7
Auto/Moto,4271604,21.6,1077690,21.0,3193915,21.8
Multietapa,2178389,11.0,296453,5.8,1881937,12.9
A demanda,862182,4.4,279859,5.5,582323,4.1
Otros Multietapa,14586,0.1,1876,0.0,12711,0.1


### Viajes en transporte público

In [22]:
particion_viajes3 = particion_viajes[particion_viajes.Modo_agg2=='Público'].copy().reset_index(drop=True)
particion_viajes3['Viajes_porc'] = (particion_viajes3['Viajes'] / particion_viajes3['Viajes'].sum() * 100).round(1)
particion_viajes3['CABA_porc'] = (particion_viajes3['CABA'] / particion_viajes3['CABA'].sum() * 100).round(1)
particion_viajes3['Conurbano_porc'] = (particion_viajes3['Conurbano'] / particion_viajes3['Conurbano'].sum() * 100).round(1)
particion_viajes3 = particion_viajes3.sort_values('Viajes', ascending=False).reset_index(drop=True)
particion_viajes3.to_csv('modos_publico2009.csv', sep=';', encoding='latin1')
particion_viajes3

,Modo,Modo_agg,Modo_agg2,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
0,Colectivo,Público,Público,5670340,66.2,1655998,70.2,4014342,64.6
1,Multietapa,Multietapa,Público,2178389,25.4,296453,12.6,1881937,30.3
2,Subte/Premetro,Público,Público,372489,4.3,359696,15.3,12793,0.2
3,Ferrocarril,Público,Público,349324,4.1,45958,1.9,303366,4.9


#### Creo sub-categorías de actividad destino y dummies para agregar a nivel persona

In [23]:
Viajes['ACTIDEST_AGG'] = Viajes.ACTIDEST.replace(list(Viajes.ACTIDEST.unique()), 
            ['Trabajo', 'Otro', 'Hogar', 'Compras', 'Estudio', 'Trabajo', 'Personal', 'Personal', 'Cuidado', 'Personal', 'Cuidado',
             'Cuidado', 'Personal', 'Personal', 'Salud', 'Personal', 'Personal', 'Personal', 'Estudio', 'Personal'])
                                                 
Viajes = pd.concat([Viajes, pd.get_dummies(Viajes.ACTIDEST_AGG)], axis=1)

Viajes = pd.concat([Viajes, pd.get_dummies(Viajes.Modo_agg2)], axis=1)

Viajes = pd.concat([Viajes, pd.get_dummies(Viajes.Zona_destino)], axis=1)

Viajes = pd.concat([Viajes, pd.get_dummies(Viajes.Zona_origen)], axis=1)

#### Agregación de viajes a nivel de Personas y agrego motivo y modo de transporte a tabla de Personas

In [24]:
Pers_agg = Viajes.groupby(['IDH', 'IDP']).agg({'Compras':'max', 'Cuidado':'max', 'Estudio':'max', 'Hogar':'max', 'Otro':'max', 'Personal':'max', 'Salud':'max',
       'Trabajo':'max', 'A demanda':'max', 'Auto/Moto':'max', 'No Motorizado':'max', 'Público':'max',
       'Destino CABA':'max', 'Destino Conurbano':'max', 'Origen CABA':'max', 'Origen Conurbano':'max'}).reset_index()
Pers_agg['Realiza_viaje'] = 1
Personas = Personas.set_index(['IDH', 'IDP']).join(Pers_agg.set_index(['IDH', 'IDP'])).reset_index()
Personas['Realiza_viaje'] = Personas['Realiza_viaje'].fillna(0)
Personas['Origen CABA'] = Personas['Realiza_viaje'].fillna(0)
Personas['Origen Conurbano'] = Personas['Realiza_viaje'].fillna(0)

In [25]:
Personas['Zona_Destino'] = Personas['Zona']
Personas.loc[(Personas.Zona == 'CABA')&(Personas['Destino Conurbano']==1), 'Zona_Destino'] = 'Conurbano'
Personas.loc[(Personas.Zona == 'Conurbano')&(Personas['Destino CABA']==1), 'Zona_Destino'] = 'CABA'

In [26]:
m1 = Personas[Personas.OCUPPRI=='Trabaja'].groupby(['Zona', 'Zona_Destino']).wt1.sum().round().astype(int).reset_index()
m1[m1.Zona!=m1.Zona_Destino]

,Zona,Zona_Destino,wt1
1,CABA,Conurbano,146465
2,Conurbano,CABA,911050


In [27]:
m2 = Personas.groupby(['Zona', 'Zona_Destino']).wt1.sum().round().astype(int).reset_index()
m1[m2.Zona!=m2.Zona_Destino]

,Zona,Zona_Destino,wt1
1,CABA,Conurbano,146465
2,Conurbano,CABA,911050


In [28]:
Personas[Personas.OCUPPRI=='Trabaja'].groupby(['Zona', 'Destino Conurbano']).wt1.sum().round().astype(int)

Zona       Destino Conurbano
CABA       0.0                  1038754
           1.0                   146465
Conurbano  0.0                    21443
           1.0                  3074237
Name: wt1, dtype: int32

# Construcción de indicadores para el análisis

In [29]:

# Población
datos = DataFrame([['Población', PoblTotal, PoblConurbano, PoblCABA]], columns=['dato', 'Total', 'Conurbano', 'CABA'])

# Viajes
datos = datos.append(DataFrame([['Viajes (partido hogar)', Viajes.wt1.sum().round().astype(int), 
                                 Viajes[Viajes.Zona!='CABA'].wt1.sum().round().astype(int), 
                                 Viajes[Viajes.Zona=='CABA'].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']), ignore_index=True)
datos = datos.append(DataFrame([['Viajes (partido origen)', Viajes.wt1.sum().round().astype(int), 
                                 Viajes[Viajes.Zona_origen=='Origen Conurbano'].wt1.sum().round().astype(int), 
                                 Viajes[Viajes.Zona_origen=='Origen CABA'].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes (partido destino)', Viajes.wt1.sum().round().astype(int), 
                                 Viajes[Viajes.Zona_destino=='Destino Conurbano'].wt1.sum().round().astype(int), 
                                 Viajes[Viajes.Zona_destino=='Destino CABA'].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Viajes en TPubl (partido hogar)', Viajes[(Viajes.Público==1)].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona!='CABA')&(Viajes.Público==1)].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona=='CABA')&(Viajes.Público==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']), ignore_index=True)
datos = datos.append(DataFrame([['Viajes en TPubl (partido origen)', Viajes[(Viajes.Público==1)].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_origen=='Origen Conurbano')&(Viajes.Público==1)].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_origen=='Origen CABA')&(Viajes.Público==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes en TPubl (partido destino)', Viajes[(Viajes.Público==1)].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_destino=='Destino Conurbano')&(Viajes.Público==1)].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_destino=='Destino CABA')&(Viajes.Público==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

# Viajes sin vuelta  ahogar
datos = datos.append(DataFrame([['Viajes (partido hogar) (sin vuelta Hogar)', Viajes[Viajes['ACTIDEST_AGG']!='Hogar'].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona!='CABA')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona=='CABA')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']), ignore_index=True)
datos = datos.append(DataFrame([['Viajes (partido origen) (sin vuelta Hogar)', Viajes[(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_origen=='Origen Conurbano')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_origen=='Origen CABA')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes (partido destino) (sin vuelta Hogar)', Viajes[(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Viajes en TPubl (partido hogar) (sin vuelta Hogar)', Viajes[(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona!='CABA')&(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona=='CABA')&(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']), ignore_index=True)
datos = datos.append(DataFrame([['Viajes en TPubl (partido origen) (sin vuelta Hogar)', Viajes[(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_origen=='Origen Conurbano')&(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_origen=='Origen CABA')&(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes en TPubl (partido destino) (sin vuelta Hogar)', Viajes[(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_destino=='Destino Conurbano')&(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                 Viajes[(Viajes.Zona_destino=='Destino CABA')&(Viajes.Público==1)&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


# Etapas
datos = datos.append(DataFrame([['Etapas', Etapas.wt1.sum().round().astype(int), Etapas[Etapas.Zona!='CABA'].wt1.sum().round().astype(int), Etapas[Etapas.Zona=='CABA'].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']), ignore_index=True)
datos = datos.append(DataFrame([['Etapas TPubl', Etapas[Etapas.Público==1].wt1.sum().round().astype(int), Etapas[(Etapas.Zona!='CABA')&(Etapas.Público==1)].wt1.sum().round().astype(int), Etapas[(Etapas.Zona=='CABA')&(Etapas.Público==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']), ignore_index=True)









# Viajes promedio por persona (promedio ponderado por fex)
v1 = Viajes.groupby(['IDH', 'IDP', 'Zona', 'wt1']).size().reset_index()
v1['pond'] = v1['wt1']*v1[0]
viajes_avg = DataFrame([['Viajes por persona (promedio)', (v1['pond'].sum()/v1['wt1'].sum()).round(2),
(v1.loc[v1.Zona=='CABA', 'pond'].sum()/v1.loc[v1.Zona=='CABA', 'wt1'].sum()).round(2),
(v1.loc[v1.Zona=='Conurbano', 'pond'].sum()/v1.loc[v1.Zona=='Conurbano', 'wt1'].sum()).round(2)]], columns=['dato', 'Total', 'Conurbano', 'CABA'])
datos = datos.append(viajes_avg)

# Viajes en TPubl promedio por persona (promedio ponderado por fex)
v1 = Viajes[Viajes.Público==1].groupby(['IDH', 'IDP', 'Zona', 'wt1']).size().reset_index()
v1['pond'] = v1['wt1']*v1[0]
viajes_tp_avg = DataFrame([['Viajes por persona en TPubl (promedio)', (v1['pond'].sum()/v1['wt1'].sum()).round(2),
(v1.loc[v1.Zona=='CABA', 'pond'].sum()/v1.loc[v1.Zona=='CABA', 'wt1'].sum()).round(2),
(v1.loc[v1.Zona=='Conurbano', 'pond'].sum()/v1.loc[v1.Zona=='Conurbano', 'wt1'].sum()).round(2)]], columns=['dato', 'Total', 'Conurbano', 'CABA'])
datos = datos.append(viajes_tp_avg)

# Personas que viajan
datos = datos.append(DataFrame([['Personas que viajan (hogar)', Personas[(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int),
                             Personas[(Personas.Zona=='Conurbano')&(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int),
                            Personas[(Personas.Zona=='CABA')&(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Personas que viajan (origen)', Personas[(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int),
                             Personas[(Personas['Origen Conurbano']==1)&(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int),
                            Personas[(Personas['Origen CABA']==1)&(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Pers q viajan (hogar-dde viajes)', (datos[datos.dato=='Viajes (partido hogar)'].Total.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].Total.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes (partido hogar)'].Conurbano.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].Conurbano.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes (partido hogar)'].CABA.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].CABA.values[0]).round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Pers q viajan (origen-dde viajes)', (datos[datos.dato=='Viajes (partido origen)'].Total.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].Total.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes (partido origen)'].Conurbano.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].Conurbano.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes (partido origen)'].CABA.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].CABA.values[0]).round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Personas que viajan en TPubl (hogar)', Personas[(Personas.Público==1)].wt1.sum().round().astype(int),
                             Personas[(Personas.Zona=='Conurbano')&(Personas.Público==1)].wt1.sum().round().astype(int),
                            Personas[(Personas.Zona=='CABA')&(Personas.Público==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Personas que viajan en TPubl (origen)', Personas[(Personas.Público==1)].wt1.sum().round().astype(int),
                             Personas[(Personas['Origen Conurbano']==1)&(Personas.Público==1)].wt1.sum().round().astype(int),
                            Personas[(Personas['Origen CABA']==1)&(Personas.Público==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Pers q viajan TP (hogar-dde viajes)', (datos[datos.dato=='Viajes en TPubl (partido hogar)'].Total.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].Total.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes en TPubl (partido hogar)'].Conurbano.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].Conurbano.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes en TPubl (partido hogar)'].CABA.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].CABA.values[0]).round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Pers q viajan TP (origen-dde viajes)', (datos[datos.dato=='Viajes en TPubl (partido origen)'].Total.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].Total.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes en TPubl (partido origen)'].Conurbano.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].Conurbano.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes en TPubl (partido origen)'].CABA.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].CABA.values[0]).round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


datos = datos.append(DataFrame([['Trabajadores', Personas[Personas.OCUPPRI=='Trabaja'].wt1.sum().round().astype(int),
                           Personas[(Personas.Zona=='Conurbano')&(Personas.OCUPPRI=='Trabaja')].wt1.sum().round().astype(int),
                           Personas[(Personas.Zona=='CABA')&(Personas.OCUPPRI=='Trabaja')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Trabajadores T.Publ', Personas[(Personas.OCUPPRI=='Trabaja')&(Personas.Público==1)].wt1.sum().round().astype(int),
                                 Personas[(Personas.Zona=='Conurbano')&(Personas.OCUPPRI=='Trabaja')&(Personas.Público==1)].wt1.sum().round().astype(int),
                                 Personas[(Personas.Zona=='CABA')&(Personas.OCUPPRI=='Trabaja')&(Personas.Público==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


datos = datos.append(pd.DataFrame([['Personas por trabajo de Conurbano a',
Personas[(Personas.Zona=='Conurbano')&(Personas.OCUPPRI=='Trabaja')].wt1.sum().round().astype(int),
Personas[(Personas.Zona=='Conurbano')&(Personas.Zona_Destino=='Conurbano')&(Personas.OCUPPRI=='Trabaja')&(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int),
Personas[(Personas.Zona=='Conurbano')&(Personas.Zona_Destino=='CABA')&(Personas.OCUPPRI=='Trabaja')&(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


datos = datos.append(pd.DataFrame([['Personas por trabajo de CABA a',
Personas[(Personas.Zona=='CABA')&(Personas.OCUPPRI=='Trabaja')].wt1.sum().round().astype(int),
Personas[(Personas.Zona=='CABA')&(Personas.Zona_Destino=='Conurbano')&(Personas.OCUPPRI=='Trabaja')&(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int),
Personas[(Personas.Zona=='CABA')&(Personas.Zona_Destino=='CABA')&(Personas.OCUPPRI=='Trabaja')&(Personas.Realiza_viaje==1)].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


# Rate Viajes/Personas
v1 = Viajes[Viajes.ACTIDEST_AGG=='Trabajo'].groupby(['IDH', 'IDP', 'wt1']).size().reset_index()
v1['tot_temp'] = v1['wt1'] * v1[0]
viajes_pers_trabajo = (v1.wt1.sum() / v1.tot_temp.sum()).round(2)
datos = datos.append(pd.DataFrame([['Rate Viajes/Persona (Trabajo)', viajes_pers_trabajo, viajes_pers_trabajo, viajes_pers_trabajo]], columns=['dato', 'Total', 'Conurbano', 'CABA'] ))

v1 = Viajes[(Viajes.ACTIDEST_AGG!='Hogar')&(Viajes.ACTIDEST_AGG!='Trabajo')].groupby(['IDH', 'IDP', 'wt1']).size().reset_index()
v1['tot_temp'] = v1['wt1'] * v1[0]
viajes_pers_otros = (v1.wt1.sum() / v1.tot_temp.sum()).round(2)
datos = datos.append(pd.DataFrame([['Rate Viajes/Persona (Otros)', viajes_pers_otros, viajes_pers_otros, viajes_pers_otros]], columns=['dato', 'Total', 'Conurbano', 'CABA'] ))

# Construyo matriz de origen y destino de viajes total, motivo trabajo y motivo otros

datos = datos.append(pd.DataFrame([['Viajes de CABA a', Viajes[Viajes.Zona=='CABA'].wt1.sum().round().astype(int), Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino Conurbano')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino CABA')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes de Conurbano a', Viajes[Viajes.Zona=='Conurbano'].wt1.sum().round().astype(int), Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino Conurbano')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino CABA')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes de CABA a (sin vuelta Hogar)', Viajes[(Viajes.Zona=='CABA')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes de Conurbano a (sin vuelta Hogar)', Viajes[(Viajes.Zona=='Conurbano')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


datos = datos.append(pd.DataFrame([['Viajes trabajo de CABA a', Viajes[(Viajes.Zona=='CABA')&(Viajes['ACTIDEST_AGG']=='Trabajo')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']=='Trabajo')].wt1.sum().round().astype(int), Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']=='Trabajo')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes trabajo de Conurbano a', Viajes[(Viajes.Zona=='Conurbano')&(Viajes['ACTIDEST_AGG']=='Trabajo')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']=='Trabajo')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']=='Trabajo')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes otros de CABA a', Viajes[(Viajes.Zona=='CABA')&(Viajes['ACTIDEST_AGG']!='Trabajo')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']!='Trabajo')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']!='Trabajo')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes otros de Conurbano a', Viajes[(Viajes.Zona=='Conurbano')&(Viajes['ACTIDEST_AGG']!='Trabajo')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']!='Trabajo')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']!='Trabajo')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes otros de CABA a (sin vuelta Hogar)', Viajes[(Viajes.Zona=='CABA')&(Viajes['ACTIDEST_AGG']!='Trabajo')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']!='Trabajo')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='CABA')&(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']!='Trabajo')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes otros de Conurbano a (sin vuelta Hogar)', Viajes[(Viajes.Zona=='Conurbano')&(Viajes['ACTIDEST_AGG']!='Trabajo')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino Conurbano')&(Viajes['ACTIDEST_AGG']!='Trabajo')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int), 
                                    Viajes[(Viajes.Zona=='Conurbano')&(Viajes.Zona_destino=='Destino CABA')&(Viajes['ACTIDEST_AGG']!='Trabajo')&(Viajes['ACTIDEST_AGG']!='Hogar')].wt1.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


# Cantidad de autos
Hogares['Cant_autos'] = Hogares.CANAUTOS.replace(['No tiene', '1.0', '2.0', '3.0', '4.0'], [0,1,2,3,4])
Hogares['Hog_Cant_autos'] = Hogares['Cant_autos'] * Hogares['wt1']

datos = datos.append(pd.DataFrame([['Cantidad de autos (totales en hogares)', (Hogares.Hog_Cant_autos.sum()).round(0).astype(int), 
                                    (Hogares[Hogares.Zona=='Conurbano'].Hog_Cant_autos.sum()).round(0).astype(int), 
                                    (Hogares[Hogares.Zona=='CABA'].Hog_Cant_autos.sum()).round(0).astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


datos = datos.reset_index(drop=True)

# Construyo campos de porcentajes 
datos['Conurbano_porc'] = (datos['Conurbano'] / datos['Total'] * 100).round(2)
datos['CABA_porc'] = (datos['CABA'] / datos['Total'] * 100).round(2)

datos['Total_porc_pobl'] = (datos.Total / datos[datos.dato=='Población'].Total.values * 100).round(2)
datos['Conurbano_porc_pobl'] = (datos.Conurbano / datos[datos.dato=='Población'].Conurbano.values * 100).round(2)
datos['CABA_porc_pobl'] = (datos.CABA / datos[datos.dato=='Población'].CABA.values * 100).round(2)



datos = datos.fillna(0)

datos.to_csv('datos_2009.csv', sep=';', encoding='latin1')
datos

,dato,Total,Conurbano,CABA,Conurbano_porc,CABA_porc,Total_porc_pobl,Conurbano_porc_pobl,CABA_porc_pobl
0,Población,12985885.00,9996947.00,2988938.00,76.98,23.02,100.00,100.00,100.00
1,Viajes (partido hogar),19767983.00,14630452.00,5137532.00,74.01,25.99,152.23,146.35,171.88
2,Viajes (partido origen),19767983.00,13525812.00,6242172.00,68.42,31.58,152.23,135.30,208.84
3,Viajes (partido destino),19767983.00,13508312.00,6259672.00,68.33,31.67,152.23,135.12,209.43
4,Viajes en TPubl (partido hogar),8570543.00,6212439.00,2358104.00,72.49,27.51,66.00,62.14,78.89
5,Viajes en TPubl (partido origen),8570543.00,5374250.00,3196293.00,62.71,37.29,66.00,53.76,106.94
6,Viajes en TPubl (partido destino),8570543.00,5360357.00,3210186.00,62.54,37.46,66.00,53.62,107.40
7,Viajes (partido hogar) (sin vuelta Hogar),10470340.00,7686288.00,2784052.00,73.41,26.59,80.63,76.89,93.15
8,Viajes (partido origen) (sin vuelta Hogar),10470340.00,7557464.00,2912876.00,72.18,27.82,80.63,75.60,97.46
9,Viajes (partido destino) (sin vuelta Hogar),10470340.00,6571637.00,3898704.00,62.76,37.24,80.63,65.74,130.44


In [30]:
v = Viajes[Viajes.Modo_agg2!='No Motorizado'].groupby(['Zona', 'Zona_destino', 'Modo_agg2']).wt1.sum().round().reset_index()
v['tot'] = v.groupby(['Zona', 'Zona_destino']).wt1.transform('sum') 
v['tot'] = (v['wt1'] / v['tot'] * 100).round(1)
v[v.Modo_agg2=='Público']



,Zona,Zona_destino,Modo_agg2,wt1,tot
2,CABA,Destino CABA,Público,2274465.0,64.6
5,CABA,Destino Conurbano,Público,83640.0,42.7
8,Conurbano,Destino CABA,Público,935721.0,72.8
11,Conurbano,Destino Conurbano,Público,5276718.0,60.5


# Motivos de los viajes

In [31]:
dest = Viajes.groupby('ACTIDEST_AGG').wt1.sum().round().astype(int).reset_index()
dest['porc_tot'] = (dest['wt1'] / dest['wt1'].sum() * 100).round(1)
dest['porc_tot_sinHogar'] = 0 
dest.loc[dest.ACTIDEST_AGG!='Hogar', 'porc_tot_sinHogar'] = (dest.loc[dest.ACTIDEST_AGG!='Hogar', 'wt1'] / dest.loc[dest.ACTIDEST_AGG!='Hogar', 'wt1'].sum() * 100).round(1)

dest = dest.rename(columns={'wt1': 'Total', 'ACTIDEST_AGG': 'Motivo'})

dest['Total_porc_pobl'] = (dest.Total / datos[datos.dato=='Población'].Total.values * 100).round(2)


dest.to_csv('motivos2009.csv', sep=';')
dest.sort_values('porc_tot_sinHogar', ascending=False)

,Motivo,Total,porc_tot,porc_tot_sinHogar,Total_porc_pobl
7,Trabajo,3944538,20.0,37.7,30.38
2,Estudio,2535687,12.8,24.2,19.53
5,Personal,1511715,7.6,14.4,11.64
1,Cuidado,1186472,6.0,11.3,9.14
0,Compras,759786,3.8,7.3,5.85
6,Salud,443626,2.2,4.2,3.42
4,Otro,88516,0.4,0.8,0.68
3,Hogar,9297643,47.0,0.0,71.60


In [32]:
Viajes.head()

,IDH,IDP,IDV,partido,fraccion,LOCALIDA,ORDEN,ACTIORIG,ACTIDEST,HORAINI,HORAFIN,TIPOTRAN,HORASALI,MINSALID,HORALLEG,MINLLEGA,FRECUENC,wt1,z_frecuenc_imput,TT_PARTIDO_ORIGEN,TT_LOCALIDAD_ORIGEN,TT_PARTIDO_DESTINO,TT_LOCALIDAD_DESTINO,Zona,Zona_destino,Zona_origen,Modo,Modo_agg,Modo_agg2,ACTIDEST_AGG,Compras,Cuidado,Estudio,Hogar,Otro,Personal,Salud,Trabajo,A demanda,Auto/Moto,No Motorizado,Público,Destino CABA,Destino Conurbano,Origen CABA,Origen Conurbano
0,2.0,12345.0,104669.0,La Matanza,01,RAMOS MEJIA,1.0,Hogar,Lugar de trabajo,1.001042e+10,1.001042e+10,0,7.0,45.0,9.0,0.0,99999,182.836868,4.829143,La Matanza,RAMOS MEJIA,Cap. Fed. DE 01,SAN NICOLAS,Conurbano,Destino CABA,Origen Conurbano,Multietapa,Multietapa,Público,Trabajo,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1
1,2.0,12345.0,104668.0,La Matanza,01,RAMOS MEJIA,1.0,Lugar de trabajo,Lugar de trabajo,1.001044e+10,1.001044e+10,0,13.0,0.0,13.0,30.0,99999,182.836868,2.384141,Cap. Fed. DE 01,SAN NICOLAS,Cap. Fed. DE 06,SAN CRISTOBAL,Conurbano,Destino CABA,Origen CABA,Subte/Premetro,Público,Público,Trabajo,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0
2,2.0,12345.0,104667.0,La Matanza,01,RAMOS MEJIA,1.0,Lugar de trabajo,Otro,1.001046e+10,1.001046e+10,0,19.0,0.0,19.0,15.0,99999,182.836868,2.384141,Cap. Fed. DE 06,SAN CRISTOBAL,Cap. Fed. DE 02,BALVANERA,Conurbano,Destino CABA,Origen CABA,A pie,No Motorizado,No Motorizado,Otro,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0
3,2.0,12345.0,104666.0,La Matanza,01,RAMOS MEJIA,1.0,Otro,Hogar,1.001046e+10,1.001046e+10,0,19.0,30.0,20.0,0.0,99999,182.836868,1.685239,Cap. Fed. DE 02,BALVANERA,La Matanza,RAMOS MEJIA,Conurbano,Destino Conurbano,Origen CABA,Ferrocarril,Público,Público,Hogar,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0
4,11.0,12367.0,104665.0,Alte. Brown,02,JOSE MARMOL,2.0,Asunto laboral,Compras,1.001043e+10,1.001043e+10,0,10.0,20.0,10.0,40.0,99999,348.047668,2.384141,Alte. Brown,JOSE MARMOL,Alte. Brown,ADROGUE,Conurbano,Destino Conurbano,Origen Conurbano,Automovil/conductor,Auto/Moto,Auto/Moto,Compras,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1


In [33]:
print('motivos de viaje')
print((Viajes[Viajes.ACTIDEST_AGG!='Hogar'].groupby('ACTIDEST_AGG').wt1.sum() / Viajes[Viajes.ACTIDEST_AGG!='Hogar'].wt1.sum()).round(2).sort_values(ascending=False))
print('')
print('motivos de viaje Conurbano/CABA')
print((Viajes[(Viajes.ACTIDEST_AGG!='Hogar')&(Viajes.Zona_origen=='Origen Conurbano')&(Viajes.Zona_destino=='Destino CABA')].groupby('ACTIDEST_AGG').wt1.sum() / Viajes[(Viajes.ACTIDEST_AGG!='Hogar')&(Viajes.Zona_origen=='Origen Conurbano')&(Viajes.Zona_destino=='Destino CABA')].wt1.sum()).round(2).sort_values(ascending=False))

print('motivos de viaje CABA/Conurbano')
print((Viajes[(Viajes.ACTIDEST_AGG!='Hogar')&(Viajes.Zona_origen=='Origen CABA')&(Viajes.Zona_destino=='Destino Conurbano')].groupby('ACTIDEST_AGG').wt1.sum() / Viajes[(Viajes.ACTIDEST_AGG!='Hogar')&(Viajes.Zona_origen=='Origen CABA')&(Viajes.Zona_destino=='Destino Conurbano')].wt1.sum()).round(2).sort_values(ascending=False))

motivos de viaje
ACTIDEST_AGG
Trabajo     0.38
Estudio     0.24
Personal    0.14
Cuidado     0.11
Compras     0.07
Salud       0.04
Otro        0.01
Name: wt1, dtype: float64

motivos de viaje Conurbano/CABA
ACTIDEST_AGG
Trabajo     0.66
Estudio     0.12
Personal    0.10
Salud       0.05
Cuidado     0.05
Compras     0.02
Otro        0.01
Name: wt1, dtype: float64
motivos de viaje CABA/Conurbano
ACTIDEST_AGG
Trabajo     0.59
Personal    0.21
Estudio     0.08
Cuidado     0.06
Salud       0.03
Compras     0.03
Otro        0.02
Name: wt1, dtype: float64


In [34]:
# Promedio de viajes por persona (motivo trabajo, otros, total sin hogar, total)
Viajes[Viajes.ACTIDEST_AGG=='Trabajo'].groupby(['IDH', 'IDP']).size().mean().round(2), \
Viajes[(Viajes.ACTIDEST_AGG!='Trabajo')&(Viajes.ACTIDEST_AGG!='Hogar')].groupby(['IDH', 'IDP']).size().mean().round(2), \
Viajes[(Viajes.ACTIDEST_AGG!='Hogar')].groupby(['IDH', 'IDP']).size().mean().round(2), \
Viajes.groupby(['IDH', 'IDP']).size().mean().round(2)

(1.09, 1.22, 1.24, 2.34)

# Vehiculos

In [35]:
# Vehiculos (autos y motos)
Hogares[(Hogares.z_tieneciclomo=='Si')|(Hogares.z_tienemoto=='Si')|(Hogares.z_tieneauto=='Si')].wt1.sum().round().astype(int), \
(Hogares[(Hogares.z_tieneciclomo=='Si')|(Hogares.z_tienemoto=='Si')|(Hogares.z_tieneauto=='Si')].wt1.sum() / Hogares.wt1.sum()).round(2)

(1529088, 0.37)

In [36]:
# Cantidad de autos
Hogares['Cant_autos'] = Hogares.CANAUTOS.replace(['No tiene', '1.0', '2.0', '3.0', '4.0'], [0,1,2,3,4])
Hogares['Hog_Cant_autos'] = Hogares['Cant_autos'] * Hogares['wt1']
(Hogares.Hog_Cant_autos.sum()).round(0).astype(int), \
(Hogares.Hog_Cant_autos.sum() / Hogares.wt1.sum()).round(2)

(1621245, 0.39)